In [76]:
import requests
from bs4 import BeautifulSoup

In [78]:
url="https://www.flipkart.com/search?q=sarees&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&p%5B%5D=facets.rating%255B%255D%3D1%25E2%2598%2585%2B%2526%2Babove&page=1"
page=requests.get(url)
page

<Response [200]>

In [80]:
page.text

'<!doctype html><html lang="en"><head><link href="https://rukminim2.flixcart.com" rel="preconnect"/><link rel="stylesheet" href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/atlas.chunk.8dd48d.css"/><link rel="stylesheet" href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app_modules.chunk.c48a12.css"/><link rel="stylesheet" href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app.chunk.e4e719.css"/><meta http-equiv="Content-type" content="text/html; charset=utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=Edge"/><meta property="fb:page_id" content="102988293558"/><meta property="fb:admins" content="658873552,624500995,100000233612389"/><link rel="shortcut icon" href="https:///www/promos/new/20150528-140547-favicon-retina.ico"/><link type="application/opensearchdescription+xml" rel="search" href="/osdd.xml?v=2"/><meta property="og:type" content="website"/><meta name="og_site_name" property="og:site_name" content="Flipka

In [82]:
soup=BeautifulSoup(page.text)

In [84]:
soup

<!DOCTYPE html>
<html lang="en"><head><link href="https://rukminim2.flixcart.com" rel="preconnect"/><link href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/atlas.chunk.8dd48d.css" rel="stylesheet"/><link href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app_modules.chunk.c48a12.css" rel="stylesheet"/><link href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app.chunk.e4e719.css" rel="stylesheet"/><meta content="text/html; charset=utf-8" http-equiv="Content-type"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="102988293558" property="fb:page_id"/><meta content="658873552,624500995,100000233612389" property="fb:admins"/><link href="https:///www/promos/new/20150528-140547-favicon-retina.ico" rel="shortcut icon"/><link href="/osdd.xml?v=2" rel="search" type="application/opensearchdescription+xml"/><meta content="website" property="og:type"/><meta content="Flipkart.com" name="og_site_name" property="og:sit

In [88]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

url = "https://www.flipkart.com/search?q=sarees&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&p%5B%5D=facets.rating%255B%255D%3D1%25E2%2598%2585%2B%2526%2Babove"
brands = []
designs = []
types = []
fabrics = []
Prices = []
Discounts = []
colors = []
ActualPrice = []

# Updated pattern for fabric extraction
fabric_pattern = r'\b(?:Pure|Art)?\s?(?:Silk Blend|Cotton Silk|Art Silk|Cotton Blend|Silk|Cotton|Georgette|Chiffon|Crepe|Net|Satin|Tissue|Jacquard|Supernet|Lycra Blend|Banarasi|Chanderi)\b'

def extract_fabrics(text):
    matches = re.findall(fabric_pattern, text, re.IGNORECASE)
    unique_matches = list(dict.fromkeys([match.strip().title() for match in matches if match.strip()]))
    return ', '.join(unique_matches)

# Function to fetch saree names
def fetch_saree_links(soup):
    saree_links = soup.find_all('a', class_='WKTcLC')
    saree_names = [link.get('title') for link in saree_links if link.get('title')]
    return saree_names

for i in range(1, 20):
    print(url + "&page=" + str(i))
    response = requests.get(url + "&page=" + str(i))
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Brands
    b = soup.find_all('div', class_='syl9yP')
    for i in b:
        brands.append(i.text)
    
    # Saree links
    links = fetch_saree_links(soup)
    
    # Extract design, type, and fabric
    design_pattern = r"^\S+"
    type_pattern = r"^\S+\s+(\S+)"
    fabric_name_pattern = r"(\S+(?:\s\S+)*)(?=\s*Saree)"
    
    for link in links:
        try:
            design = re.search(design_pattern, link).group()
            type_ = re.search(type_pattern, link).group(1)
            fabric = re.search(fabric_name_pattern, link)
            fabric_text = fabric.group(1) if fabric else ""
            fabrics.append(extract_fabrics(fabric_text))
            designs.append(design)
            types.append(type_)
        except:
            continue
    
    # Prices
    p = soup.find_all('div', 'Nx9bqj')
    for i in p:
        x = int(re.sub(r'[^\d]', '', i.text))
        Prices.append(x)

    # Discounts
    d = soup.find_all('span')
    for tag in d:
        if '% off' in tag.text:
            match = re.search(r'(\d+)%', tag.text)
            if match:
                Discounts.append(int(match.group(1)))

    # Colors
    c = soup.find_all('div', 'Br9IW+')
    for i in c:
        x = i.text.strip()
        colors.append(x)

    # Actual Prices
    da = soup.find_all('div', class_="yRaY8j")
    for item in da:
        text = item.get_text()
        amount = re.search(r'₹\s*([\d,]+)', text)
        if amount:
            ActualPrice.append(amount.group(1).replace(',', ''))

# Now you can create your dataframe with all the lists:
# df = pd.DataFrame({
#     'Brand': brands,
#     'Design': designs,
#     'Type': types,
#     'Fabric': fabrics,
#     'Price': Prices,
#     'Discount': Discounts,
#     'Color': colors,
#     'ActualPrice': ActualPrice
# })

# Uncomment the above block to generate a DataFrame

print("Brands:", len(brands))
print("Designs:", len(designs))
print("Types:", len(types))
print("Fabrics:", len(fabrics))
print("Prices:", len(Prices))
print("Discounts:", len(Discounts))
print("Colors:", len(colors))
print("ActualPrices:", len(ActualPrice))


https://www.flipkart.com/search?q=sarees&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&p%5B%5D=facets.rating%255B%255D%3D1%25E2%2598%2585%2B%2526%2Babove&page=1
https://www.flipkart.com/search?q=sarees&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&p%5B%5D=facets.rating%255B%255D%3D1%25E2%2598%2585%2B%2526%2Babove&page=2
https://www.flipkart.com/search?q=sarees&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&p%5B%5D=facets.rating%255B%255D%3D1%25E2%2598%2585%2B%2526%2Babove&page=3
https://www.flipkart.com/search?q=sarees&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&p%5B%5D=facets.rating%255B%255D%3D1%25E2%2598%2585%2B%2526%2Babove&page=4
https://www.flipkart.com/search?q=sarees&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&p%5B%5D=facets.rating%255B%255D%3D1%25E2%2598%2585%2B%2526%2Babove&page=5
https://www.flipkart.com/search?q=sarees&otracker=search&otracker

In [90]:
import pandas as pd

# Create a dictionary with the column names as keys and lists as values
data = {
    'Brand': brands,
    'Designs': designs,
    'Types': types,
    'Fabrics': fabrics,
    'Colors': colors,
    'Discounts': Discounts,
    'Discount_Price': Prices,
    'ActualPrice':ActualPrice
}

# Create the DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df
#df = df.reset_index().rename(columns={'index': 'ID'})
#df

,Brand,Designs,Types,Fabrics,Colors,Discounts,Discount_Price,ActualPrice
0,KanjiQueen,Woven,Kanjivaram,"Cotton Silk, Silk Blend","Beige, Maroon",90,499,4999
1,KanjiQueen,Checkered,Jamdani,"Silk Blend, Cotton Blend","White, Maroon",90,499,4999
2,jihana fab,Printed,Kanjivaram,"Georgette, Silk Blend",Maroon,62,299,799
3,Kalpana creation,Printed,Bandhani,Georgette,Multicolor,89,414,3999
4,Leelavati,Printed,Daily,Georgette,"Pack of 2, Multicolor",75,489,1999
...,...,...,...,...,...,...,...,...
755,shivdhara,Printed,Patola,Georgette,Multicolor,53,375,799
756,SWORNOF,Woven,Banarasi,"Banarasi, Silk Blend, Jacquard","Dark Blue, Maroon",74,459,1800
757,WearEZee,Digital,"Print,","Satin, Crepe",Purple,60,1178,2999
758,ETHNIC SAREES Bharat,"Printed,","Checkered,",Pure Cotton,Brown,65,449,1299


In [92]:
import os
os.getcwd()

'C:\\Users\\Dell'

In [96]:
df.to_csv("EDA_PRO.csv")

# Data cleaning

In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Brand           760 non-null    object
 1   Designs         760 non-null    object
 2   Types           760 non-null    object
 3   Fabrics         760 non-null    object
 4   Colors          760 non-null    object
 5   Discounts       760 non-null    int64 
 6   Discount_Price  760 non-null    int64 
 7   ActualPrice     760 non-null    object
dtypes: int64(2), object(6)
memory usage: 47.6+ KB


In [100]:
df['ActualPrice'] = df['ActualPrice'].astype(str)                          # Ensure strings
df['ActualPrice'] = df['ActualPrice'].str.replace(',', '', regex=False)  # Remove commas
df['ActualPrice'] = df['ActualPrice'].str.extract('(\d+)')               # Extract only digits
df['ActualPrice'] = pd.to_numeric(df['ActualPrice'], errors='coerce')    # Convert to int safely


<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Dell\AppData\Local\Temp\ipykernel_1728\3189208728.py:3: SyntaxWarning: invalid escape sequence '\d'
  df['ActualPrice'] = df['ActualPrice'].str.extract('(\d+)')               # Extract only digits


In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Brand           760 non-null    object
 1   Designs         760 non-null    object
 2   Types           760 non-null    object
 3   Fabrics         760 non-null    object
 4   Colors          760 non-null    object
 5   Discounts       760 non-null    int64 
 6   Discount_Price  760 non-null    int64 
 7   ActualPrice     760 non-null    int64 
dtypes: int64(3), object(5)
memory usage: 47.6+ KB


In [104]:
df.shape

(760, 8)

In [106]:
df.columns

Index(['Brand', 'Designs', 'Types', 'Fabrics', 'Colors', 'Discounts',
       'Discount_Price', 'ActualPrice'],
      dtype='object')

In [108]:
df.isnull().sum()

Brand             0
Designs           0
Types             0
Fabrics           0
Colors            0
Discounts         0
Discount_Price    0
ActualPrice       0
dtype: int64